In [1]:
import pickle
import random
import numpy as np
import networkx as nx
from sklearn.decomposition import PCA

import torch

from smartsampling import *
from evaluation import *

In [2]:
seed = 1
device = "cuda" if torch.cuda.is_available() else "cpu"

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device=="cuda":
    torch.cuda.manual_seed(seed)

In [3]:
node_features = pickle.load(open('data/cora/node_features.pkl', 'rb'))
text_features = node_features.toarray()

pca = PCA(n_components=100)
pca.fit(text_features)
emb_features = pca.transform(text_features)

nnodes = text_features.shape[0]
device = "cuda" if torch.cuda.is_available() else "cpu"

## Only 1 node sampled from neighbors each walk

In [4]:
p_space = [9e50] #[0.2, 0.5, 1, 2, 5]
q_space = [9e50] #[0.2, 0.5, 1, 2, 5]
c_space = [1] #[1, 2, 3, 4, 5]

nsamplers = int(nnodes/1)
nwalk = 100
nnega = 5
penalty = 3e-4
dropout = 0.4

lr = 0.005
weight_decay = 0.0005

ratio = 0.1
nfold = 5

In [5]:
lp_train = pickle.load(open('data/cora/lp_train.pkl', 'rb'))
lp_test = pickle.load(open('data/cora/lp_test.pkl', 'rb'))

X_train, y_train, X_test, y_test = pickle.load(open('simulation/train_test.pkl', 'rb'))

# node_adjs = nx.from_scipy_sparse_matrix(pickle.load(open('data/cora/node_adjs.pkl', 'rb')))
# node_labels = pickle.load(open('data/cora/node_labels.pkl', 'rb'))

In [6]:
model = SmartSampling(text_features, emb_features, p_space, q_space, c_space, 
                      nnodes, nsamplers, nnega, lr, weight_decay, dropout, device, 
                      lp_train, X_train, y_train)

In [7]:
embeddings = model.train(0, penalty)
# nc_acc, nc_f1 = nc_evaluate(embeddings, node_labels)
# print('Walk: {}'.format(0), nc_acc, nc_f1)
lp_acc, lp_f1 = lp_evaluate(embeddings, lp_test)
print("Walk: {}".format(0), lp_acc, lp_f1)
    
for i in range(1, nwalk+1):
    embeddings = model.train(1, penalty)
    lp_acc, lp_f1 = lp_evaluate(embeddings, lp_test)
    print("Walk: {}".format(i), lp_acc, lp_f1)

Walk: 0 0.6803364879074658 0.7179962894248609
Epoch: 1, Loss: 0.7076, Acc: 0.3871, F1: 0.5582, Gain: 1.0163 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 10.0013s
Walk: 1 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.7059, Acc: 0.3871, F1: 0.5582, Gain: 0.8477 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.7069s
Walk: 2 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.7031, Acc: 0.3871, F1: 0.5582, Gain: 0.8344 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.6887s
Walk: 3 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.7014, Acc: 0.3871, F1: 0.5582, Gain: 0.8301 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.6669s
Walk: 4 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.6996, Acc: 0.3871, F1: 0.5582, Gain: 0.8242 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.7248s
Walk: 5 0.49947423764458465 0.6661991584852734
Epoch: 1, Loss: 0.6969, Acc: 0.3871, F1: 0.5582, Gain: 0.8220 || Nonneighbor_Avg: 0.0000

Epoch: 1, Loss: 0.6519, Acc: 0.3961, F1: 0.5617, Gain: 0.6493 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.9360s
Walk: 49 0.5026288117770767 0.6671358198451794
Epoch: 1, Loss: 0.6507, Acc: 0.3961, F1: 0.5617, Gain: 0.6431 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.5920s
Walk: 50 0.5036803364879074 0.667605633802817
Epoch: 1, Loss: 0.6508, Acc: 0.3963, F1: 0.5616, Gain: 0.6469 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.8956s
Walk: 51 0.501577287066246 0.6666666666666665
Epoch: 1, Loss: 0.6494, Acc: 0.3975, F1: 0.5623, Gain: 0.6343 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.6376s
Walk: 52 0.5047318611987381 0.6685432793807178
Epoch: 1, Loss: 0.6481, Acc: 0.3978, F1: 0.5624, Gain: 0.6357 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.6724s
Walk: 53 0.5057833859095688 0.6676096181046676
Epoch: 1, Loss: 0.6508, Acc: 0.3977, F1: 0.5623, Gain: 0.6392 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.6578s
Walk: 54 0.

Epoch: 1, Loss: 0.6482, Acc: 0.3993, F1: 0.5628, Gain: 0.6404 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.5483s
Walk: 97 0.5078864353312302 0.6699576868829337
Epoch: 1, Loss: 0.6471, Acc: 0.4015, F1: 0.5640, Gain: 0.6507 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.8354s
Walk: 98 0.5078864353312302 0.6694915254237289
Epoch: 1, Loss: 0.6495, Acc: 0.4000, F1: 0.5632, Gain: 0.6452 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.8153s
Walk: 99 0.5057833859095688 0.6685472496473907
Epoch: 1, Loss: 0.6469, Acc: 0.4017, F1: 0.5638, Gain: 0.6501 || Nonneighbor_Avg: 0.0000, Neighbor_Avg: 1.0000, Time: 9.3305s
Walk: 100 0.508937960042061 0.6694975230007076
